In [1]:
import bs4 
import re 
import os 
import json
import requests
import datetime
import urllib
from dateutil import parser
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np 
import sqlite3
pd.set_option('max_colwidth', 300)
pd.set_option('max_columns', 100)

## Functions

### Fun Fact.. Dealnews uses an API 

In [2]:
def get_dealnews_feed(link):
    headers = {'Authorization': 'DN jxqfz29pbv9xpWSYnmJX'}
    dealnews = requests.get(link, headers=headers)
    assert(dealnews.status_code == 200), f"Status Code={dealnews.status_code}...Error:\n\n{dealnews.content}"
    as_json = json.loads(dealnews.content)
    return(as_json)

### Understanding API Options

In [3]:
def _parse_api_deal_types(raw_api_response):
    ## Deal Types
    keeper_elems = ['name', 'count', 'url', 'short_name']
    groups = raw_api_response['deal_types']
    key_group_info = []
    for group in groups:
        key_group_info.append([group[x] for x in keeper_elems])
    deal_type_df = pd.DataFrame(key_group_info, columns = keeper_elems)
    deal_type_df = deal_type_df.rename({'url': 'id_number'}, axis =1)
    deal_type_df['id_name'] = 'deal_type_id'
    return(deal_type_df)

In [4]:
def _parse_api_categories(raw_api_response):
    ## Categories
    keeper_elems = ['name', 'count', 'category_id', 'short_name']
    groups = raw_api_response['categories']
    key_group_info = []
    for group in groups:
        key_group_info.append([group[x] for x in keeper_elems])
    category_df = pd.DataFrame(key_group_info, columns = keeper_elems)
    category_df = category_df.rename({'category_id': 'id_number'}, axis =1)
    category_df['id_name'] = 'category_id'
    return(category_df)

In [5]:
def _parse_api_brands(raw_api_response):
    ## Brands
    keeper_elems = ['name', 'count', 'brand_id', 'short_name']
    groups = raw_api_response['brands']
    key_group_info = []
    for group in groups:
        key_group_info.append([group[x] for x in keeper_elems])
    brand_df = pd.DataFrame(key_group_info, columns = keeper_elems)
    brand_df = brand_df.rename({'brand_id': 'id_number'}, axis =1)
    brand_df['id_name'] = 'brand_id'
    return(brand_df)

In [6]:
def _parse_api_vendors(raw_api_response):
    ## Vendors
    keeper_elems = ['name', 'count', 'vendor_id', 'short_name']
    groups = raw_api_response['vendors']
    key_group_info = []
    for group in groups:
        key_group_info.append([group[x] for x in keeper_elems])
    vendor_df = pd.DataFrame(key_group_info, columns = keeper_elems)
    vendor_df = vendor_df.rename({'vendor_id': 'id_number'}, axis =1)
    vendor_df['id_name'] = 'vendor_id'
    return(vendor_df)

In [7]:
def _parse_api_facet_groups(raw_api_response):
    ## Facet Groups 
    keeper_elems = ['name', 'count', 'facet_id', 'short_name']
    facet_group = raw_api_response['facet_groups']
    key_group_info = []
    # This one has a group within each group (2 levels before getting to the mapping)
    for facet in facet_group:
        groups = facet['facets']
        facet_category_name = facet['name']
        for group in groups:
            group_list = [facet_category_name + ' : ' + group[x] if x == 'name' 
                          else group[x] for x in keeper_elems]
            key_group_info.append(group_list)
    facet_df = pd.DataFrame(key_group_info, columns = keeper_elems)
    facet_df = facet_df.rename({'facet_id': 'id_number'}, axis =1)
    facet_df['id_name'] = 'facet_id'
    return(facet_df)

### Summarize Options 

In [8]:
def get_summary_of_api_options(df):
    out = {}
    out['Offers Sum'] = df['count'].sum()
    out['Group Count'] = df.shape[0]
    df = df.sort_values(by='count', ascending=False)
    most_common = df.iloc[0]
    out['Most Common Name'] = most_common['name']
    out['Most Common Count'] = most_common['count']
    out['Most Common ID'] = most_common['id_number']
    
    out_series = pd.Series(out)
    return(out_series)

## Begin Running it 

### Parse API Categories with Counts

In [9]:
con = sqlite3.connect('dealnews.db')
cursor = con.cursor()

In [10]:
link = 'https://api.dealnews.com/content?facet_ids=1780&count=70'
raw_api_response = get_dealnews_feed(link)

In [11]:
deal_types_df = _parse_api_deal_types(raw_api_response)
vendor_df = _parse_api_vendors(raw_api_response)
facet_df = _parse_api_facet_groups(raw_api_response)
brand_df = _parse_api_brands(raw_api_response)
category_df = _parse_api_categories(raw_api_response)


df_type = {
#     "deal_type" : deal_types_df, # not really helpful with the API calls 
"vendor" : vendor_df,
"facet_group" : facet_df,
"brand" : brand_df,
"category" : category_df}

all_dfs = pd.concat(df_type)
all_dfs.head(30)

name  count  id_number            short_name  \
vendor 0                 Amazon    716        313                Amazon   
       1                   eBay    145         50                  eBay   
       2             Home Depot     63        958            Home Depot   
       3                 Macy's     39        288                Macy's   
       4             Sam's Club     33        857            Sam's Club   
       5                 Kohl's     30       1009                Kohl's   
       6           Ace Hardware     30       1320          Ace Hardware   
       7         Nordstrom Rack     23      41081        Nordstrom Rack   
       8                Wayfair     22      38875               Wayfair   
       9          Northern Tool     20        562         Northern Tool   
       10               Staples     19        274               Staples   
       11               Walmart     19        321               Walmart   
       12                  Nike     19       1186                  Nike   
       13                Target     17        318                Target   
       14                   REI     17        549                   REI   
       15                    HP     17        786                    HP   
       16                Reebok     15        908                Reebok   
       17         moobibear.com     14      54815         moobibear.com   
       18              Old Navy     13        362              Old Navy   
       19       Microsoft Store     12       3626       Microsoft Store   
       20  Best Choice Products     12      42941  Best Choice Products   
       21               Adorama     11       1409               Adorama   
       22                 Tanga     11       2216                 Tanga   
       23         5.11 Tactical     11      39715         5.11 Tactical   
       24                Proozy     11      41921                Proozy   
       25               Costway     13      46658               Costway   
       26                 Petco      9        474                 Petco   
       27             Dell Home      9        638             Dell Home   
       28    PepsiCo via Amazon      9      46889    PepsiCo via Amazon   
       29       B&H Photo Video      8        973                   B&H   

             id_name  
vendor 0   vendor_id  
       1   vendor_id  
       2   vendor_id  
       3   vendor_id  
       4   vendor_id  
       5   vendor_id  
       6   vendor_id  
       7   vendor_id  
       8   vendor_id  
       9   vendor_id  
       10  vendor_id  
       11  vendor_id  
       12  vendor_id  
       13  vendor_id  
       14  vendor_id  
       15  vendor_id  
       16  vendor_id  
       17  vendor_id  
       18  vendor_id  
       19  vendor_id  
       20  vendor_id  
       21  vendor_id  
       22  vendor_id  
       23  vendor_id  
       24  vendor_id  
       25  vendor_id  
       26  vendor_id  
       27  vendor_id  
       28  vendor_id  
       29  vendor_id

In [12]:
summary1 = all_dfs.reset_index().groupby('level_0')\
    .apply(get_summary_of_api_options)
summary1.index = summary1.index.rename('API Category Name')
summary1

,Offers Sum,Group Count,Most Common Name,Most Common Count,Most Common ID
API Category Name,,,,,
brand,1002,481,Apple,23,13
category,2199,30,Home & Garden,753,196
facet_group,9742,34,Popularity Rank : Popularity: 1/5,4428,1786
vendor,2016,356,Amazon,716,313


### Save to Database

In [13]:
all_dfs['_dt_pulled'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
all_dfs.to_sql('Category Info', con, if_exists='append')

summary1['_dt_pulled'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
summary1.to_sql('Category Summary', con, if_exists='append')

/home/malcolm/main/lib/python3.6/site-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


## Parse One API Section Request

In [14]:

def parse_easy_elems(content):
    content_keys_to_keep = [
         'id',
         'headline',
         'brief_headline',
         'summary',
         'secondary_summary',
         'key_attribute',
         'brief_notes',
         'url',
         'display_date',
         'publish_datetime_ts',
         'update_datetime_ts',
         'expiration_datetime_ts',
         'last_verified_datetime_ts',
         'editors_choice',
         'sponsored',
         'expired',
         'expires_today',
         'exclusive',
         'searchable',
         'hotness',
         'call_out',
         'call_out_comparison',
         'sub_call_out'
    ]
    out_dict = {x : content[x] for x in content_keys_to_keep}
    return(out_dict)

def parse_time_fields(content):
    elements = ['publish_datetime_ts',
         'update_datetime_ts',
         'expiration_datetime_ts',
         'last_verified_datetime_ts']
    out_dict = {}
    for x in elements:
        temp_time = content.get(x) if content.get(x) != None else 0
        out_dict[x] = datetime.datetime.fromtimestamp(temp_time)
    return(out_dict)

def parse_coupon_code(content):
    cc_list = content['coupon_code']
    cc_code = cc_list[0] if len(cc_list) > 0 else None 
    out_dict = {"Coupon Code":cc_code}
    return(out_dict)

def _parse_category(cat_dict):
    keeper_elements = ['category_id', 'name'
                      'path', 'ancestor_list' ]
    cat_elems = {x:cat_dict.get(x) for x in keeper_elements}
    return(cat_elems)

def parse_2_categories(content):
    cat_list = content['categories']
    out_dict = {}
    # Only keep 2 categories 
    if len(cat_list) < 2:
        cat_list.append({})
    for dict_num in range(2):
        temp_cat_elems = _parse_category(cat_list[dict_num])
        temp_cat_elems = {key + '_' +str(dict_num): value 
                          for key, value in temp_cat_elems.items()}
        out_dict.update(temp_cat_elems)
    
    return(out_dict)

def parse_vendor(content):
    vend_dict = content['vendor']
    keeper_elems = ['vendor_id', 'name']
    vend_dict = {x:vend_dict[x] for x in keeper_elems}
    return(vend_dict)

def _create_img_path(content):
    # Create Filename
    headline = content.get('headline')
    id1 = content.get('id')
    file_name = headline + '_' + str(id1) + '.jpg'
    file_name = file_name.replace('/', '')
    # Create Directory structure 
    date = str(datetime.datetime.now().date())
    if 'base_path' not in locals():
        base_path = os.getcwd() + '/Dealnews Images/'
    path = base_path + date + '/'
    os.makedirs(path, exist_ok=True)
    #File Save location 
    save_location = path + file_name
    return(save_location)
    
def parse_image(content):
    image_link = content.get('images').get('XXL').get('url')
    save_path = _create_img_path(content)
    
    urllib.request.urlretrieve(image_link, save_path)
    out_dict = {'Image path': save_path}
    return(out_dict)


In [15]:
def parse_dn_item(content):
    all_details = {}
    
    main_elems = parse_easy_elems(content)
    all_details.update(main_elems)
    
    time_fields = parse_time_fields(content)
    all_details.update(time_fields)
    
    cc_code = parse_coupon_code(content)
    all_details.update(cc_code)
    
    categories = parse_2_categories(content)
    all_details.update(categories)

    vendor = parse_vendor(content)
    all_details.update(vendor)
    
    image_info = parse_image(content)
    all_details.update(image_info)
    
    return(all_details)

In [16]:
def _get(colname):
    return(popularity.columns.tolist().index(colname))

In [17]:
popularity = facet_df[facet_df.name.str.contains('Popularity Rank')]
popularity['n_api_return_items'] = 70
popularity['# of times to call'] = popularity['count']/popularity['n_api_return_items']
popularity['# of times to call'] = 1 + popularity['# of times to call'].astype('int')
popularity

/home/malcolm/main/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/malcolm/main/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/malcolm/main/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

,name,count,id_number,short_name,id_name,n_api_return_items,# of times to call
0,Popularity Rank : Popularity: 5/5,112,1774,Popularity: 5/5,facet_id,70,2
1,Popularity Rank : Popularity: 4/5,557,1777,Popularity: 4/5,facet_id,70,8
2,Popularity Rank : Popularity: 3/5,2020,1780,Popularity: 3/5,facet_id,70,29
3,Popularity Rank : Popularity: 2/5,1208,1783,Popularity: 2/5,facet_id,70,18
4,Popularity Rank : Popularity: 1/5,4428,1786,Popularity: 1/5,facet_id,70,64


In [18]:
last_published_item_raw = cursor.execute("""
select `API Feed`, max(publish_datetime_ts) 
from `Dealnews Items`
group by `API Feed`
""").fetchall()
last_published_dict = {x[0]: x[1] for x in last_published_item_raw}
last_published_dict

{'Popularity Rank : Popularity: 1/5': '2021-06-16 19:01:25',
 'Popularity Rank : Popularity: 2/5': '2021-06-16 18:58:43',
 'Popularity Rank : Popularity: 3/5': '2021-06-16 18:39:02',
 'Popularity Rank : Popularity: 4/5': '2021-06-16 18:56:45',
 'Popularity Rank : Popularity: 5/5': '2021-06-16 18:58:07'}

In [20]:
page_items = []
for row in popularity.iterrows():
    # Set up 
    row = row[1]
    print("On group: \n", row)
    temp_name = row[_get('name')]
    page_num_called = 1
    id_num = row[_get('id_number')]
    last_published_item = last_published_dict[temp_name]
    
    while page_num_called <= row[_get('# of times to call')]:
        temp_link = f'https://api.dealnews.com/content?facet_ids={id_num}&count=70&page={page_num_called}'
        raw_api_response = get_dealnews_feed(temp_link)
        for content1 in tqdm(raw_api_response['content']):
            elems = parse_dn_item(content1)
            elems['API Feed'] = temp_name
            elems['API id_number'] = id_num
            elems['Page'] = page_num_called
            elems_series = pd.Series(elems)
            item_publish_time = elems_series['publish_datetime_ts'] 
            if item_publish_time > last_published_item:
                page_items.append(elems_series)
            else:
                break
        page_num_called = page_num_called + 1


On group: 
 name                  Popularity Rank : Popularity: 5/5
count                                               112
id_number                                          1774
short_name                              Popularity: 5/5
id_name                                        facet_id
n_api_return_items                                   70
# of times to call                                    2
Name: 0, dtype: object



On group: 
 name                  Popularity Rank : Popularity: 4/5
count                                               557
id_number                                          1777
short_name                              Popularity: 4/5
id_name                                        facet_id
n_api_return_items                                   70
# of times to call                                    8
Name: 1, dtype: object



On group: 
 name                  Popularity Rank : Popularity: 3/5
count                                              2020
id_number                                          1780
short_name                              Popularity: 3/5
id_name                                        facet_id
n_api_return_items                                   70
# of times to call                                   29
Name: 2, dtype: object



On group: 
 name                  Popularity Rank : Popularity: 2/5
count                                              1208
id_number                                          1783
short_name                              Popularity: 2/5
id_name                                        facet_id
n_api_return_items                                   70
# of times to call                                   18
Name: 3, dtype: object



On group: 
 name                  Popularity Rank : Popularity: 1/5
count                                              4428
id_number                                          1786
short_name                              Popularity: 1/5
id_name                                        facet_id
n_api_return_items                                   70
# of times to call                                   64
Name: 4, dtype: object


In [21]:
dn_today = pd.concat(page_items, axis =1).T
dn_today

,id,headline,brief_headline,summary,secondary_summary,key_attribute,brief_notes,url,display_date,publish_datetime_ts,update_datetime_ts,expiration_datetime_ts,last_verified_datetime_ts,editors_choice,sponsored,expired,expires_today,exclusive,searchable,hotness,call_out,call_out_comparison,sub_call_out,Coupon Code,category_id_0,namepath_0,ancestor_list_0,category_id_1,namepath_1,ancestor_list_1,vendor_id,name,Image path,API Feed,API id_number,Page
0,19312637,Woot Garage Sale,"Woot Garage Sale: Discounts on groceries, tools, electronics & more","Save on hundreds of deals including home items, electronics, computers, groceries, apparel, exercise and sports equipment, and more.","<div class=""snippet summary"" title=""Save&#x20;on&#x20;hundreds&#x20;of&#x20;deals&#x20;including&#x20;home&#x20;items,&#x20;electronics,&#x20;computers,&#x20;groceries,&#x20;apparel,&#x20;exercise&#x20;and&#x20;sports&#x20;equipment,&#x20;and&#x20;more.""> <p>Save on hundreds of deals including h...",Woot! An Amazon Company,,https://www.dealnews.com/Woot-Garage-Sale-Discounts-on-groceries-tools-electronics-more-free-shipping-w-Prime/19312637.html?iref=site-widget,Last verified 1 hr 27 min ago,2021-06-16 08:52:38,2021-06-16 20:03:23,2021-06-28 05:00:00,2021-06-16 18:35:48,False,False,False,False,False,True,5,"Discounts on groceries, tools, electronics & more",,free shipping w/ Prime,None,40,None,None,None,None,None,1692,Woot! An Amazon Company,/home/malcolm/Demo with APIs/Dealnews Images/2021-06-16/Woot Garage Sale_19312637.jpg,Popularity Rank : Popularity: 5/5,1774,1
1,19313456,"Domisl 20"" Folding Shelf Brackets 2-Pack","Domisl 20"" Folding Shelf Brackets 2-Pack for $20","Apply coupon code ""40H7YVAW"" for a savings of $13.","<div class=""body-offer body-burst""> <div class=""snippet summary"" title=""Apply&#x20;coupon&#x20;code&#x20;&quot;40H7YVAW&quot;&#x20;for&#x20;a&#x20;savings&#x20;of&#x20;&#x24;13.""> <p>Apply coupon code ""40H7YVAW"" for a savings of $13. <a target=""_blank"" href=""https://www.dealnews.com/lw/click.htm...",Amazon,"Free shipping. Sold Faihts via Amazon. <br><b>Features:</b> includes mounting screws, 400-lbs. maximum load capacity and made of high-quality 304 stainless steel.",https://www.dealnews.com/Domisl-20-Folding-Shelf-Brackets-2-Pack-for-20-free-shipping/19313456.html?iref=site-widget,Last verified 1 hr 22 min ago,2021-06-16 16:29:13,2021-06-16 20:02:45,2021-06-18 15:17:00,2021-06-16 18:41:24,False,False,False,False,False,True,5,$20,$33,free shipping,40H7YVAW,862,None,196,None,None,None,313,Amazon,"/home/malcolm/Demo with APIs/Dealnews Images/2021-06-16/Domisl 20"" Folding Shelf Brackets 2-Pack_19313456.jpg",Popularity Rank : Popularity: 5/5,1774,1
2,19309286,"""Options Trading Crash Course"" Kindle eBook","""Options Trading Crash Course"" Kindle eBook: Free",Save $10 off the digital list price.,"<div class=""snippet summary"" title=""Save&#x20;&#x24;10&#x20;off&#x20;the&#x20;digital&#x20;list&#x20;price.""> <p>Save $10 off the digital list price. <a target=""_blank"" href=""https://www.dealnews.com/lw/click.html?20,2,19309286,iref=site-widget"">Shop Now at Amazon</a> </p> </div> <div class=""sni...",Amazon,<br><b>Features:</b> 96 pages.,https://www.dealnews.com/Options-Trading-Crash-Course-Kindle-eBook-Free/19309286.html?iref=site-widget,Last verified 2 hr 50 min ago,2021-06-16 04:19:43,2021-06-16 19:53:38,1970-01-01 00:00:00,2021-06-16 17:12:34,False,False,False,False,False,True,5,Free,,None,None,554,None,"178,177,553",None,None,None,313,Amazon,"/home/malcolm/Demo with APIs/Dealnews Images/2021-06-16/""Options Trading Crash Course"" Kindle eBook_19309286.jpg",Popularity Rank : Popularity: 5/5,1774,1
3,19308422,Mobil 1 5W-30 Extended Performance Full Synthetic Motor Oil 5-Quart Bottle,Mobil 1 5W-30 Extended Performance Full Synthetic Motor Oil 5-Quart Bottle for $9,It's $15 under list price.,"<div class=""snippet summary"" title=""Redeem&#x20;this&#x20;""> <p>Redeem this <a target=""_blank"" href=""https://acbincentives

## Examine and Export

In [22]:
dn_today[['headline', 'brief_headline', 'summary', 'key_attribute'
          , 'call_out', 'call_out_comparison']]

,headline,brief_headline,summary,key_attribute,call_out,call_out_comparison
0,Woot Garage Sale,"Woot Garage Sale: Discounts on groceries, tools, electronics & more","Save on hundreds of deals including home items, electronics, computers, groceries, apparel, exercise and sports equipment, and more.",Woot! An Amazon Company,"Discounts on groceries, tools, electronics & more",
1,"Domisl 20"" Folding Shelf Brackets 2-Pack","Domisl 20"" Folding Shelf Brackets 2-Pack for $20","Apply coupon code ""40H7YVAW"" for a savings of $13.",Amazon,$20,$33
2,"""Options Trading Crash Course"" Kindle eBook","""Options Trading Crash Course"" Kindle eBook: Free",Save $10 off the digital list price.,Amazon,Free,
3,Mobil 1 5W-30 Extended Performance Full Synthetic Motor Oil 5-Quart Bottle,Mobil 1 5W-30 Extended Performance Full Synthetic Motor Oil 5-Quart Bottle for $9,It's $15 under list price.,Walmart,$9.37,$24
4,ASICS Men's Upcourt 2 Shoes,ASICS Men's Upcourt 2 Shoes for $18,"Apply coupon code ""SHOE10"" to save.",Shoebacca,$18,$65
...,...,...,...,...,...,...
8465,OSP Home Furnishings Barton Writing Desk,OSP Home Furnishings Barton Writing Desk for $172,It's $258 under list price.,Kohl's,$172,$430
8466,Levi's Girls' 711 Skinny Jeans,Levi's Girls' 711 Skinny Jeans for $13,It's $31 under list price.,Macy's,$13,$44
8467,$50 Chuck E Cheese Gift Card,$50 Chuck E Cheese Gift Card for $38 for members,It's $13 under list price.,Sam's Club,$38 for members,$50
8468,$10 Amora Coffee Gift Card,$10 Amora Coffee Gift Card: free w/ $40 purchase,Earn a free gift with a $40 or more purchase of coffee and tea.,Amora Coffee,free w/ $40 purchase,


In [23]:
dn_today['key_attribute'].value_counts()

Amazon                   2845
eBay                     1042
Sam's Club                424
Petco                     338
Kohl's                    311
                         ... 
Udemy                       2
Krispy Kreme                2
AlphabetDeal                2
Dick's Sporting Goods       2
Carhartt                    2
Name: key_attribute, Length: 100, dtype: int64

In [24]:
print(dn_today.columns)

Index(['id', 'headline', 'brief_headline', 'summary', 'secondary_summary',
       'key_attribute', 'brief_notes', 'url', 'display_date',
       'publish_datetime_ts', 'update_datetime_ts', 'expiration_datetime_ts',
       'last_verified_datetime_ts', 'editors_choice', 'sponsored', 'expired',
       'expires_today', 'exclusive', 'searchable', 'hotness', 'call_out',
       'call_out_comparison', 'sub_call_out', 'Coupon Code', 'category_id_0',
       'namepath_0', 'ancestor_list_0', 'category_id_1', 'namepath_1',
       'ancestor_list_1', 'vendor_id', 'name', 'Image path', 'API Feed',
       'API id_number', 'Page'],
      dtype='object')


In [25]:
dn_today.head()

,id,headline,brief_headline,summary,secondary_summary,key_attribute,brief_notes,url,display_date,publish_datetime_ts,update_datetime_ts,expiration_datetime_ts,last_verified_datetime_ts,editors_choice,sponsored,expired,expires_today,exclusive,searchable,hotness,call_out,call_out_comparison,sub_call_out,Coupon Code,category_id_0,namepath_0,ancestor_list_0,category_id_1,namepath_1,ancestor_list_1,vendor_id,name,Image path,API Feed,API id_number,Page
0,19312637,Woot Garage Sale,"Woot Garage Sale: Discounts on groceries, tools, electronics & more","Save on hundreds of deals including home items, electronics, computers, groceries, apparel, exercise and sports equipment, and more.","<div class=""snippet summary"" title=""Save&#x20;on&#x20;hundreds&#x20;of&#x20;deals&#x20;including&#x20;home&#x20;items,&#x20;electronics,&#x20;computers,&#x20;groceries,&#x20;apparel,&#x20;exercise&#x20;and&#x20;sports&#x20;equipment,&#x20;and&#x20;more.""> <p>Save on hundreds of deals including h...",Woot! An Amazon Company,,https://www.dealnews.com/Woot-Garage-Sale-Discounts-on-groceries-tools-electronics-more-free-shipping-w-Prime/19312637.html?iref=site-widget,Last verified 1 hr 27 min ago,2021-06-16 08:52:38,2021-06-16 20:03:23,2021-06-28 05:00:00,2021-06-16 18:35:48,False,False,False,False,False,True,5,"Discounts on groceries, tools, electronics & more",,free shipping w/ Prime,None,40,None,None,None,None,None,1692,Woot! An Amazon Company,/home/malcolm/Demo with APIs/Dealnews Images/2021-06-16/Woot Garage Sale_19312637.jpg,Popularity Rank : Popularity: 5/5,1774,1
1,19313456,"Domisl 20"" Folding Shelf Brackets 2-Pack","Domisl 20"" Folding Shelf Brackets 2-Pack for $20","Apply coupon code ""40H7YVAW"" for a savings of $13.","<div class=""body-offer body-burst""> <div class=""snippet summary"" title=""Apply&#x20;coupon&#x20;code&#x20;&quot;40H7YVAW&quot;&#x20;for&#x20;a&#x20;savings&#x20;of&#x20;&#x24;13.""> <p>Apply coupon code ""40H7YVAW"" for a savings of $13. <a target=""_blank"" href=""https://www.dealnews.com/lw/click.htm...",Amazon,"Free shipping. Sold Faihts via Amazon. <br><b>Features:</b> includes mounting screws, 400-lbs. maximum load capacity and made of high-quality 304 stainless steel.",https://www.dealnews.com/Domisl-20-Folding-Shelf-Brackets-2-Pack-for-20-free-shipping/19313456.html?iref=site-widget,Last verified 1 hr 22 min ago,2021-06-16 16:29:13,2021-06-16 20:02:45,2021-06-18 15:17:00,2021-06-16 18:41:24,False,False,False,False,False,True,5,$20,$33,free shipping,40H7YVAW,862,None,196,None,None,None,313,Amazon,"/home/malcolm/Demo with APIs/Dealnews Images/2021-06-16/Domisl 20"" Folding Shelf Brackets 2-Pack_19313456.jpg",Popularity Rank : Popularity: 5/5,1774,1
2,19309286,"""Options Trading Crash Course"" Kindle eBook","""Options Trading Crash Course"" Kindle eBook: Free",Save $10 off the digital list price.,"<div class=""snippet summary"" title=""Save&#x20;&#x24;10&#x20;off&#x20;the&#x20;digital&#x20;list&#x20;price.""> <p>Save $10 off the digital list price. <a target=""_blank"" href=""https://www.dealnews.com/lw/click.html?20,2,19309286,iref=site-widget"">Shop Now at Amazon</a> </p> </div> <div class=""sni...",Amazon,<br><b>Features:</b> 96 pages.,https://www.dealnews.com/Options-Trading-Crash-Course-Kindle-eBook-Free/19309286.html?iref=site-widget,Last verified 2 hr 50 min ago,2021-06-16 04:19:43,2021-06-16 19:53:38,1970-01-01 00:00:00,2021-06-16 17:12:34,False,False,False,False,False,True,5,Free,,None,None,554,None,"178,177,553",None,None,None,313,Amazon,"/home/malcolm/Demo with APIs/Dealnews Images/2021-06-16/""Options Trading Crash Course"" Kindle eBook_19309286.jpg",Popularity Rank : Popularity: 5/5,1774,1
3,19308422,Mobil 1 5W-30 Extended Performance Full Synthetic Motor Oil 5-Quart Bottle,Mobil 1 5W-30 Extended Performance Full Synthetic Motor Oil 5-Quart Bottle for $9,It's $15 under list price.,"<div class=""snippet summary"" title=""Redeem&#x20;this&#x20;""> <p>Redeem this <a target=""_blank"" href=""https://acbincentives

In [26]:
dn_today['_dt_pulled'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
dn_today.to_sql('Dealnews Items', con, index=False, if_exists='append')

/home/malcolm/main/lib/python3.6/site-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,
